In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3 as sql
sns.set()
%matplotlib inline

In [ ]:
conn = sql.connect('../data/test.db')

In [ ]:
np.random.seed(1897)
df_1 = pd.DataFrame({'Id':np.arange(1,21),
                     'ProdId': ['A','B','C','D']*5,
                     'Length': np.random.randint(20,40,size=20),
                     'Height': np.random.randint(10,35,size=20)})
df_2 = pd.DataFrame({'Id':np.random.randint(10,size=100),
                     'Date':pd.to_timedelta(np.random.randint(0,1000,size=100), unit='d')+pd.datetime(2015,1,1),
                     'Cat_1': np.array(['A','B','C','D'])[np.random.randint(4,size=100)],
                     'Cat_2': np.array(['S','T','Q','R'])[np.random.randint(4,size=100)]})

# Assignment 5.1
Load the sheets `Customers` and `Products` from the Excel file `data/master_data.xlsx`. Store the data from the `Customers` sheet in `cust_df` and the data from the `Products` sheet in `prod_df`.
<details><summary>Hint 1</summary>
    <p>
        The pandas function `read_excel` has an option `sheet_name`, that let's one specify which sheets to extract
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        The pandas function `read_excel` returns a dictionary with the sheet names as keys
    </p>
</details>

In [ ]:
tmp = pd.read_excel('../data/master_data.xlsx',sheet_name=['Customers','Products'])
cust_df = tmp['Customers']
prod_df = tmp['Products']

# Assignment 5.2
The CSV file `data/bad_form.csv` follows a rather odd format:
* The columns are separated by `|`
* There are several comment lines. Each comment is indicated by `//`
* True values in the Boolean columns are marked by `Yes`
* False values in the Boolean columns are marked by `Nope`
* Decimals are indicated by `,`
* The string columns contain characters in `latin-1` encoding
* In column **B** the missing values are indicated by `Missing`
* In column **C** the missing values are indicated by `Not here`
* The first 5 lines in the file is merely a header - should not be loaded
* The last 10 lines make out the footer - should not be loaded

Load the file in a DataFrame with the column `Id` as index.
<details><summary>Hint 1</summary>
    <p>
        Look into the documentation for parameters `sep`, `comment`, `true_values`, `false_values`, `index_col`, `skiprows`, `skipfooter`, `na_values`, `encoding` and `decimal` for the pandas function `read_csv`.
    </p>
</details>

In [ ]:
pd.read_csv('../data/bad_form.csv',
                 sep='|',
                 engine='python',
                 comment='//',
                 true_values=['Yes'],
                 false_values=['Nope'],
                 index_col='Id',
                 skiprows=5,
                 skipfooter=10,
                 na_values={'B':'Missing','C':'Not here'},
                 encoding='latin-1',
                 decimal=',')

# Assignment 5.3
Write the content of the DataFrame `df_1` into the table `product_dimensions` in the database `test.db`. The connection to the database has already been set up in the variable `conn`.
<details><summary>Hint 1</summary>
    <p>
        Use the `to_sql` method for the DataFrame to write it to the database
    </p>
</details>

In [ ]:
df_1.to_sql('product_dimensions', conn)

# Assignment 5.4
Compute the average area of each product in `product_dimensions` using a SQL statement. Then proceed to load the data in two lines at a time, and print out the result.
<details><summary>Hint 1</summary>
    <p>
        The pandas function `read_sql` can be used to parse a SQL statement to a database and store the result into a DataFrame
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Setting the parameter `chunksize` controls the number of records returned at a time
    </p>
</details>
<details><summary>Hint 3</summary>
    <p>
        With a set `chunksize` you can subsequently loop over the reader object, that is returned from `read_sql`
    </p>
</details>

In [ ]:
reader = pd.read_sql('select ProdId, avg(Length*Height) from product_dimensions group by ProdId', conn, chunksize=2)

for chunk in reader:
    print(chunk)

# Assignment 5.5
For each `ID` extract the records up until the first time a `B` or a `C` appears in the `Cat_1` column, or a `R` or a `S` appears in the `Cat_2` column. The `Date` column indicates the order of the observations.
<details><summary>Hint 1</summary>
    <p>
        The `groupby` method preserves the order of the records as they had in the original DataFrame
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Applying the `cummax` method to a Boolean column returns 0 for every record, until the first True value is reached. From that point on it returns a 1.
    </p>
</details>
<details><summary>Hint 3</summary>
    <p>
        Using the index to merge the results onto the original DataFrame makes it possible to select the rows with value 0 in the new column.
    </p>
</details>

In [ ]:
tmp = df_2.merge(df_2
                 .assign(event=((df_2['Cat_1'].isin(['B', 'C']))|
                                (df_2['Cat_2'].isin(['R', 'S']))))
                 .sort_values(['Date'])
                 .groupby('Id')['event']
                 .cummax()
                 .to_frame(),
                 right_index=True,
                 left_index=True).sort_values(['Id','Date'])
tmp[tmp['event']==0].drop('event', axis=1)